In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.datasets as d_sets
from torch.utils.data import DataLoader as d_loader
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F
import os
import os.path as osp
import logging
from collections import OrderedDict
import json
from datetime import datetime
import os
import math
import numpy as np

from torchvision.utils import make_grid
import numpy as np
import torch
from torch.utils.data import Dataset

import logging
from re import split
from vit_pytorch import ViT

In [2]:
!nvidia-smi

Tue Nov 25 14:26:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090 D      Off |   00000000:01:00.0 Off |                  Off |
| 31%   47C    P0            110W /  425W |    1102MiB /  24564MiB |     26%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch

# 检查可用的 GPU 数量
print("Available GPUs count:", torch.cuda.device_count())

# 打印每个 GPU 的详细信息
for i in range(torch.cuda.device_count()):
    print("GPU", i, ":", torch.cuda.get_device_name(i))

Available GPUs count: 8
GPU 0 : NVIDIA GeForce RTX 4090 D
GPU 1 : NVIDIA GeForce RTX 4090 D
GPU 2 : NVIDIA GeForce RTX 4090 D
GPU 3 : NVIDIA GeForce RTX 4090 D
GPU 4 : NVIDIA GeForce RTX 4090 D
GPU 5 : NVIDIA GeForce RTX 4090 D
GPU 6 : NVIDIA GeForce RTX 4090 D
GPU 7 : NVIDIA GeForce RTX 4090 D


In [4]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

2025-11-25 14:26:14.031626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764080774.048102 1536562 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764080774.053142 1536562 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764080774.068149 1536562 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764080774.068164 1536562 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764080774.068168 1536562 computation_placer.cc:177] computation placer alr

In [5]:
import torch
torch.cuda.set_device(3)

In [6]:
def tensor2numpy(tensor, mean, std, mean_input_wind_lr, std_input_wind_lr):
    '''
    Converts a torch Tensor into a numpy array and de-normalizes it using given mean and std
    Input: 3D(1,H,W) or 4D(1,1,H,W)
    Output: 3D(1,H,W) or 4D(1,1,H,W), float32
    '''
    if tensor.dim() == 3 and tensor.size(0) == 1:
        tensor = tensor.squeeze(0)  # Remove the first dimension (C, H, W)
        tensor = tensor.float().cpu()
        
        # De-normalize using the given mean and std
        tensor = tensor * std + mean
        
        img_np = tensor.numpy()
        
        
    
    elif tensor.dim() == 4 and tensor.size(0) == 1 and tensor.size(1) == 1:
        tensor = tensor.squeeze(0).squeeze(0)  # Remove the first two dimensions (H, W)
        tensor = tensor.float().cpu()
        
        # De-normalize using the given mean and std
        tensor = tensor * std + mean
        
        img_np = tensor.numpy()
        
    elif tensor.dim() == 4 and tensor.size(0) == 1 and tensor.size(1) == 2:
        # Extract the third channel
        tensor = tensor.squeeze(0)  # Remove the first dimension (3, H, W)
        tensor = tensor[1, :, :]  # Extract the third channel (H, W)
        tensor = tensor.float().cpu()
        
        # De-normalize using the given mean_input_wind_lr and std_input_wind_lr
        tensor = tensor * std_input_wind_lr + mean_input_wind_lr
        
        img_np = tensor.numpy()
        
       
    
    else:
        raise TypeError('Only support 3D tensor with first dimension as 1 or 4D tensor with first two dimensions as 1. But received with dimension: {:d} and size: {}'.format(tensor.dim(), tensor.size()))
    
    return img_np.astype(np.float32)

def save_img(img, img_path):
    '''
    Save an image as a .npy file.
    '''
    np.save(img_path, img)


def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    max_value = max(np.max(img1), np.max(img2))
    return 20 * math.log10(max_value / math.sqrt(mse))

def calculate_mse(img1, img2):
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    return mse


In [7]:
import numpy as np
import torch
from torch.utils.data import Dataset

class LRHR_dataset(Dataset):
    def __init__(self, input_file, output_file):
        """
        初始化数据集类
        :param input_file: 包含输入数据的npy文件路径（combined_data.npy）
        :param output_file: 包含输出数据的npy文件路径（filtered_combined_wind_speed_usa.npy）
        """
        # 加载数据
        self.input_data = np.load(input_file)
        self.output_data = np.load(output_file)

        # 对输入数据进行标准化
        self.normalized_input = np.zeros_like(self.input_data)
        for i in range(self.input_data.shape[1]):  # 对每个变量进行循环
            mean = np.mean(self.input_data[:, i, :, :], axis=0, keepdims=True)
            std = np.std(self.input_data[:, i, :, :], axis=0, keepdims=True)
            if np.any(std == 0):  # 如果标准差中有任何一个为零，则跳过该变量
                print(f"Variable {i} has at least one std=0, skipping normalization for this variable.")
                self.normalized_input[:, i, :, :] = self.input_data[:, i, :, :]
            else:
                self.normalized_input[:, i, :, :] = (self.input_data[:, i, :, :] - mean) / std
            if i == 1:
                self.mean_i2 = mean
                self.std_i2 = std
        # 对输出数据进行标准化
        self.mean_output = np.mean(self.output_data, axis=0, keepdims=True)
        self.std_output = np.std(self.output_data, axis=0, keepdims=True)
        self.normalized_output = (self.output_data - self.mean_output) / self.std_output

    def __len__(self):
        """
        返回数据集中的样本数
        """
        return self.input_data.shape[0]

    def __getitem__(self, index):
        """
        根据索引获取数据集中的样本
        :param index: 数据的索引值
        """
        input_sample = self.normalized_input[index]
        output_sample = self.normalized_output[index]

        # 将NumPy数组转换为torch tensors
        input_tensor = torch.from_numpy(input_sample).float()
        output_tensor = torch.from_numpy(output_sample[np.newaxis, :, :]).float()

        return input_tensor, output_tensor
    
    def get_output_mean_std(self):
        """
        返回输出数据的均值和标准差
        """
        return self.mean_output, self.std_output
    def get_input_i2_mean_std(self):
        """
        返回input_data中i=3的均值和标准差
        """
        return self.mean_i2, self.std_i2
# 使用示例
# dataset = WindSpeedDataset('combined_data.npy', 'filtered_combined_wind_speed_usa.npy')
# input_sample, output_sample = dataset[0]


In [8]:
import torch
import torch.nn as nn

# 1. 通用封装：把任意 vit-pytorch 模型当作回归 backbone
class ViTRegressor(nn.Module):
    def __init__(self, backbone: nn.Module, image_size=64, out_ch=1):
        super().__init__()
        self.backbone = backbone
        self.H = image_size
        self.W = image_size
        self.out_ch = out_ch

    def forward(self, x):
        # x: B x 2 x 64 x 64
        b = x.size(0)
        y = self.backbone(x)  # B x (out_ch * H * W)
        return y.view(b, self.out_ch, self.H, self.W)
    



In [9]:
class Args:
    def __init__(self, batch_size=16, test_batch_size=1, epochs=160, lr=0.0001, cuda=1, threads=16, seed=123,large_kernel_size = 9,small_kernel_size = 3,n_channels = 32,n_blocks = 16):
        
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.epochs = epochs
        self.lr = lr
        self.cuda = cuda
        self.threads = threads
        self.seed = seed
        self.large_kernel_size =  large_kernel_size  # 第一层卷积和最后一层卷积的核大小
        self.small_kernel_size = small_kernel_size   # 中间层卷积的核大小
        self.n_channels = n_channels        # 中间层通道数
        self.n_blocks = n_blocks         # 残差模块数量
        #self.scaling_factor = scaling_factor
opt = Args()

print(opt)
print(opt.batch_size)
print(opt.n_channels)

16
32


In [10]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, random_split

use_cuda = opt.cuda
if use_cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(opt.seed)
if use_cuda:
    torch.cuda.set_device(4)
    torch.cuda.manual_seed(opt.seed)
    

input_file_trainset = "/data/hanzhe/math_5410_program/temperature_downscaling/dataset/input_only_2var_trainset.npy"
output_file_trainset = "/data/hanzhe/math_5410_program/temperature_downscaling/dataset/mswxda_trainset_from_nc.npy"
input_file_testset = "/data/hanzhe/math_5410_program/temperature_downscaling/dataset/input_only_2var_testset.npy"
output_file_testset = "/data/hanzhe/math_5410_program/temperature_downscaling/dataset/mswxda_testset_from_nc.npy"


train_dataset = LRHR_dataset(input_file_trainset,output_file_trainset)
test_dataset = LRHR_dataset(input_file_testset,output_file_testset)

train_mean_output,train_std_output = train_dataset.get_output_mean_std()
train_mean_input, train_std_input = train_dataset.get_input_i2_mean_std()
test_mean_output, test_std_output = test_dataset.get_output_mean_std()
test_mean_input, test_std_input = test_dataset.get_input_i2_mean_std()

training_data_loader = DataLoader(dataset=train_dataset, num_workers=opt.threads, batch_size=opt.batch_size, shuffle=True)
for batch_datas, batch_labels in training_data_loader:
    print(batch_datas.size(),batch_labels.size())
testing_data_loader = DataLoader(dataset=test_dataset, num_workers=opt.threads, batch_size=opt.test_batch_size, shuffle=False)

Variable 0 has at least one std=0, skipping normalization for this variable.
Variable 0 has at least one std=0, skipping normalization for this variable.
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.Size([16, 2, 64, 64]) torch.Size([16, 1, 64, 64])
torch.

In [11]:
from __future__ import print_function

from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

import torch
from torch import nn
import torchvision
import math

from vit_pytorch import SimpleViT

def build_simplevit_regressor():
    vit_backbone = SimpleViT(
        image_size = 64,
        patch_size = 4,
        num_classes = 64 * 64,
        dim = 256,
        depth = 6,
        heads = 8,
        mlp_dim = 512,
        channels = 2
    )
    return ViTRegressor(vit_backbone, image_size=64, out_ch=1)

srcnn = build_simplevit_regressor()


criterion = nn.MSELoss()


if(use_cuda):
    srcnn.cuda()
    criterion = criterion.cuda()

optimizer = optim.Adam(srcnn.parameters(),lr=opt.lr)
result_path = '/data/hanzhe/math_5410_program/temperature_downscaling/simpleViT/results'

def train(epoch):
    epoch_loss = 0
    srcnn.train()
    for iteration, batch in enumerate(training_data_loader, 1):
        input, target = Variable(batch[0]), Variable(batch[1])
        if use_cuda:
            input = input.cuda()
            target = target.cuda()

        optimizer.zero_grad()
        #print ("input shape = " , input.shape)
        #print ("target shape = ", target.shape)
        model_out = srcnn(input)
        #print ("model_out shape =" , model_out.shape)
        loss = criterion(model_out, target)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        writer.add_scalar('Loss/train', float(loss), epoch)

        print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))




def test(epoch):
    #avg_psnr = 0
    srcnn.eval()
    val_loss = 0
    avg_psnr = 0
    avg_mse = 0
    current_step = 0
    with torch.no_grad():
        for batch in testing_data_loader:
            current_step +=1
            input, target = Variable(batch[0]), Variable(batch[1])
            if use_cuda:
                input = input.cuda()
                target = target.cuda()

            prediction = srcnn(input)
            loss = criterion(prediction, target)
            val_loss += loss.item()
            writer.add_scalar('Loss/test', float(loss), epoch)
            
            
            sr_image = prediction.detach().cpu()
            hr_image = target.detach().cpu()
            lr_image = input.detach().cpu()
            #print("sr shape:",sr_image.shape)
            #print("hr shape:",hr_image.shape)
            #print("lr shape:",lr_image.shape)
            
            
            sr_img = tensor2numpy(sr_image,test_mean_output,test_std_output,test_mean_input,test_std_input) # uint8
            hr_img = tensor2numpy(hr_image,test_mean_output,test_std_output,test_mean_input,test_std_input)  # uint8
            lr_img = tensor2numpy(lr_image,test_mean_output,test_std_output,test_mean_input,test_std_input) # uint8
            
            epoch_path = os.path.join(result_path, f'epoch_{epoch}')
            os.makedirs(epoch_path, exist_ok=True)
            
            # 保存图像
            save_img(hr_img, os.path.join(epoch_path, f'{epoch}_{current_step}_hr.npy'))
            save_img(sr_img, os.path.join(epoch_path, f'{epoch}_{current_step}_sr.npy'))
            save_img(lr_img, os.path.join(epoch_path, f'{epoch}_{current_step}_lr.npy'))
            
            
            psnr = calculate_psnr(sr_img, hr_img)
            mse = calculate_mse(sr_img, hr_img)
            #print(psnr)
            #print(calculate_mse(sr_img, hr_img))
            avg_psnr += psnr
            avg_mse+=mse
        print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))
        print("===> Avg. MSE: {:.4f} dB".format(avg_mse / len(testing_data_loader)))
    print("===> Epoch {} Complete test: Avg. Loss: {:.4f}".format(epoch, val_loss / len(testing_data_loader)))


def checkpoint(epoch):
    model_out_path = "model_epoch_{}.pth".format(epoch)
    torch.save(srcnn, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

for epoch in range(1, opt.epochs + 1):
    train(epoch)
    test(epoch)
    if(epoch%160==0):
        checkpoint(epoch)

===> Epoch[1](1/617): Loss: 0.7734
===> Epoch[1](2/617): Loss: 0.8595
===> Epoch[1](3/617): Loss: 1.5426
===> Epoch[1](4/617): Loss: 0.8437
===> Epoch[1](5/617): Loss: 1.1010
===> Epoch[1](6/617): Loss: 0.9463
===> Epoch[1](7/617): Loss: 1.1611
===> Epoch[1](8/617): Loss: 0.6140
===> Epoch[1](9/617): Loss: 1.1956
===> Epoch[1](10/617): Loss: 0.7535
===> Epoch[1](11/617): Loss: 1.8501
===> Epoch[1](12/617): Loss: 1.1213
===> Epoch[1](13/617): Loss: 0.7735
===> Epoch[1](14/617): Loss: 0.6402
===> Epoch[1](15/617): Loss: 1.5183
===> Epoch[1](16/617): Loss: 0.7428
===> Epoch[1](17/617): Loss: 0.8507
===> Epoch[1](18/617): Loss: 0.6479
===> Epoch[1](19/617): Loss: 0.9147
===> Epoch[1](20/617): Loss: 0.4346
===> Epoch[1](21/617): Loss: 0.5685
===> Epoch[1](22/617): Loss: 0.5858
===> Epoch[1](23/617): Loss: 0.5555
===> Epoch[1](24/617): Loss: 0.7144
===> Epoch[1](25/617): Loss: 1.4480
===> Epoch[1](26/617): Loss: 0.6240
===> Epoch[1](27/617): Loss: 0.5778
===> Epoch[1](28/617): Loss: 1.7882
=

===> Epoch 155 Complete: Avg. Loss: 0.0034
===> Avg. PSNR: 34.2611 dB
===> Avg. MSE: 0.0333 dB